In [5]:
pip install yfinance mysql-connector-python apscheduler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.8 MB/s eta 0:00:00


In [7]:
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.triggers.cron import CronTrigger
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pytz
import requests
from bs4 import BeautifulSoup
import mysql.connector
from mysql.connector import Error

# Function to get S&P 500 tickers
def get_sp500_components():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    table = soup.find('table', {'class': 'wikitable'})
    tickers = []

    for row in table.find_all('tr')[1:]:  # Exclude the header row
        columns = row.find_all('td')
        ticker = columns[0].text.strip()
        tickers.append(ticker)

    return tickers

# Function to fetch historical data
def fetch_historical_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date, interval='1m')

    if data.empty:
        print(f"No data found for {symbol} from {start_date} to {end_date}.")
        return None
    else:
        data['data_vendor_id'] = 'yahoo_finance'
        data['ticker_id'] = symbol
        data['price_date'] = data.index.strftime('%Y-%m-%d %H:%M:%S')
        current_time = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
        data['created_date'] = current_time
        data['last_updated'] = current_time

        data.reset_index(drop=True, inplace=True)
        return data

# Function to upload data to MySQL in batches
def upload_data_to_database(data, table_name, batch_size=1000):
    try:
        if data.empty:
            print("No data to upload.")
            return

        data = data.rename(columns={
            'Adj Close': 'adj_close_price',
            'Close': 'close_price',
            'High': 'high_price',
            'Low': 'low_price',
            'Open': 'open_price',
            'Volume': 'volume'
        })

        data = data[['adj_close_price', 'close_price', 'high_price', 'low_price', 'open_price',
                     'volume', 'data_vendor_id', 'ticker_id', 'price_date', 'created_date', 'last_updated']]

        data_tuples = [tuple(row) for row in data.itertuples(index=False, name=None)]

        connection = mysql.connector.connect(
            host='project-db.cbegtabjn5eh.us-east-1.rds.amazonaws.com',
            user='admin',
            password='comp4651',
            database='stock_price'
        )

        if connection.is_connected():
            print("Connected to the database")

            cursor = connection.cursor()

            insert_query = f"""
            INSERT INTO {table_name} (
                adj_close_price, close_price, high_price, low_price, open_price,
                volume, data_vendor_id, ticker_id, price_date, created_date, last_updated
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """

            total_rows = len(data_tuples)
            for i in range(0, total_rows, batch_size):
                batch = data_tuples[i:i+batch_size]
                cursor.executemany(insert_query, batch)
                connection.commit()
                print(f"Uploaded batch {i // batch_size + 1}/{(total_rows + batch_size - 1) // batch_size}")

            print(f"Uploaded all {total_rows} rows successfully!")

    except Error as e:
        print(f"Error: {e}")

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

# Function to fetch and upload data
def fetch_and_upload_data():
    utc_tz = pytz.utc
    hkt_tz = pytz.timezone('Asia/Hong_Kong')

    # Define the date range for T-7 (last 7 days)
    target_end_date = datetime.now(hkt_tz)
    target_start_date = target_end_date - timedelta(days=7)

    # Convert market opening and closing times to UTC
    start_date = utc_tz.localize(datetime(target_start_date.year, target_start_date.month, target_start_date.day, 14, 30))  # 9:30 AM EST
    end_date = utc_tz.localize(datetime(target_end_date.year, target_end_date.month, target_end_date.day, 21, 0))  # 4:00 PM EST

    sp500_tickers = get_sp500_components()

    all_data_frames = []

    for symbol in sp500_tickers:
        try:
            data_frames = []
            current_start_date = start_date

            while current_start_date < end_date:
                current_end_date = min(current_start_date + timedelta(days=8), end_date)
                historical_data = fetch_historical_data(symbol, current_start_date, current_end_date)

                if historical_data is not None:
                    if isinstance(historical_data.columns, pd.MultiIndex):
                        historical_data.columns = historical_data.columns.droplevel(1)

                    data_frames.append(historical_data)

                current_start_date = current_end_date

            if data_frames:
                combined_data = pd.concat(data_frames, ignore_index=True)
                all_data_frames.append(combined_data)

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")

    if all_data_frames:
        final_combined_data = pd.concat(all_data_frames, ignore_index=True)
        print(f"Total rows fetched: {len(final_combined_data)}")
        # print(final_combined_data.head())

        try:
            upload_data_to_database(final_combined_data, "daily_price", batch_size=1000)
        except Exception as e:
            print(f"Error uploading data to database: {e}")
    else:
        print("No data was fetched for any tickers.")

# Scheduler setup
scheduler = BlockingScheduler()

# Set up the job to run daily at 08:00 AM HKT
hkt_tz = pytz.timezone('Asia/Hong_Kong')
scheduler.add_job(fetch_and_upload_data, CronTrigger(hour=8, minute=00, timezone=hkt_tz))

print("Scheduler is running. Waiting for tasks...")
try:
    scheduler.start()
except (KeyboardInterrupt, SystemExit):
    print("Scheduler stopped.")

Scheduler is running. Waiting for tasks...
Scheduler stopped.
